<a href="https://colab.research.google.com/github/shravika123/Answer_sheet_evaluation/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from spacy.cli import download
download("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import nltk;
nltk.download('stopwords')
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

data = newsgroups_train.data
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
   for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in stop_words] for doc in texts]
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
   return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [ ]:
# data_words_nostops = remove_stopwords(data_words)
# data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams = make_bigrams(data_words)
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
data_lemmatized = lemmatization(
   data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
)

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
#  mm = gensim.corpora.MmCorpus('wiki_tfidf.mm.bz2')

In [ ]:
lsi_model = gensim.models.lsimodel.LsiModel(
   corpus=corpus, id2word=id2word, num_topics=20,chunksize=100
)

In [ ]:
lsi_model.save('_Lsi.gensim')

In [ ]:
pip install pattern

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.2 MB 1.2 MB/s 
     |████████████████████████████████| 88 kB 7.3 MB/s 
     |████████████████████████████████| 81 kB 8.3 MB/s 
     |████████████████████████████████| 5.6 MB 42.4 MB/s 
     |████████████████████████████████| 5.6 MB 52.6 MB/s 
     |████████████████████████████████| 348 kB 55.4 MB/s 
     |████████████████████████████████| 100 kB 9.6 MB/s 
     |████████████████████████████████| 4.0 MB 45.7 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332722 sha256=ff77364958e1c709cee91e19fa0f847bd2e1e40f8a285d296a8844a9b4cf68ca
  Stored in directory: /root/.cache/pip/wheels/ec/ce/8f/bccc2d04f3a25a5a1dd19165b2855ad3203975f25edd5838d6
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp38-cp38-linux_x86_64.whl size=102404 sha256=dd12318d0adc0bcda428538264f1770420077fcfd2e64c679ad1a9f4116e999c
  Stored in

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from __main__ import *
import logging
from gensim import corpora, similarities
from gensim.models import LsiModel
from nltk.corpus import stopwords
from pattern.en import sentiment

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
with open("mark scheme 2.txt") as f1:
    mark_all = f1.read()

# Upload Student Answer now.
with open('ans sheet2.txt') as f2:
    ans_all = f2.read()

anssheet = ans_all.split('~\n\n')
anssheet = [d for d in anssheet if d]

if len(anssheet) == 0:
    print("student is absent for the test !")
    exit()

markscheme = mark_all.split('~\n\n')
markscheme = [d for d in markscheme if d]

new_list_ans = []
new_list_mar = []

with open('written_ans.txt') as wa:
    fwa = wa.readlines()

written_answers = fwa[0].split(',')

with open('marks_allot.txt') as ma:
    fma = ma.readlines()

marks_allotment = []
mmm = []

for r in fma:
    t = r.strip()
    t = t.split('-')
    marks_allotment.append((t[0], t[1]))
    mmm.append(t[1])

marks_allotment = dict(marks_allotment)
max_marks = 0
for i in mmm:
    j = int(i)
    max_marks = max_marks + j

for j in written_answers:
    for ans in anssheet:
        if ans.startswith(j):
            ans = ans[2:]
            new_list_ans.append(ans)

    for mar in markscheme:
        if mar.startswith(j):
            mar = mar[2:]
            new_list_mar.append(mar)

total_marks = 0


def marking_scheme(file1):
    global documents
    documents = file1.split('.')
    documents = [d.strip() for d in documents]
    documents = [d for d in documents if d]


# with open('MarkingScheme.txt') as f2:
#    file2 = f2.read()

def answer(file2):
    global new_documents
    new_documents = file2.split('.')
    new_documents = [dd.strip() for dd in new_documents]
    new_documents = [dd for dd in new_documents if dd]


# nltk.download('stopwords')


LSI = LsiModel.load('_Lsi.gensim')


def pre_process():
    global t1, t2, index, len_doc, len_new_doc, len_temp_doc, len_temp_new_doc, dictionary, new_documents, documents, corpus
    texts = [[word for word in document.lower().split() if word not in set(stopwords.words('english'))] for document in
             documents]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    index = similarities.MatrixSimilarity(LSI[corpus])
    len_new_doc = [x for x in range(len(new_documents))]
    len_doc = [y for y in range(len(documents))]
    t1 = [x for x in range(len(new_documents))]
    t2 = [y for y in range(len(documents))]
    len_temp_new_doc = len_new_doc
    len_temp_doc = len_doc


def process_sim():
    global len_new_doc, sim_array, index, dictionary, new_documents, documents
    for sent in len_new_doc[:]:
        vec_bow = dictionary.doc2bow(new_documents[sent].lower().split())
        vec_lsi = LSI[vec_bow]
        sims = index[vec_lsi]
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        lg = []
        for o, p in sims:
            if p == 0.0:
                lg.append(p)
        if lg == [0.0] * len(documents):
            t1.remove(sent)
            len_new_doc.remove(sent)
        else:
            sim_array.append(sims)


def process_zero():
    global len_doc, indx
    for c in len_doc:
        max_sim = 0
        count = -1
        for m in sim_array:
            count = count + 1
            for e, k in m:
                if e == c:
                    if k > max_sim:
                        max_sim = k
                        indx = len_new_doc[count]
        avg_sim.append((c, indx, max_sim))


def process_one():
    global len_new_doc, len_temp_new_doc, len_temp_doc, len_doc, final_sim
    for n in len_new_doc:
        tem = 0
        added = None
        for a, b, c in avg_sim:
            if b == n:
                if c > tem:
                    tem = c
                    added = (a, b, c)
        if added is not None:
            final_sim.append(added)
    len_temp_new_doc = [w for q, w, e in final_sim]
    len_temp_doc = [r for r, t, y in final_sim]
    len_new_doc = [x for x in t1 if x not in len_temp_new_doc]
    len_doc = [y for y in t2 if y not in len_temp_doc]


def process_sim1():
    global len_doc, sim_array, index, dictionary, new_documents, documents
    sim_array = []
    for sent in len_new_doc[:]:
        vec_bow = dictionary.doc2bow(new_documents[sent].lower().split())
        vec_lsi = LSI[vec_bow]
        sims = index[vec_lsi]
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        lg = []
        for o, p in sims:
            if p == 0.0:
                lg.append(p)
        if lg == [0.0] * len(documents):
            t1.remove(sent)
            len_new_doc.remove(sent)
        else:
            sim_array.append(sims)


def process_zero1():
    global len_new_doc, indx
    for c in len_doc:
        max_sim = 0
        count = -1
        indx = None
        for m in sim_array:
            count = count + 1
            for e, k in m:
                if e == c:
                    if k > max_sim:
                        max_sim = k
                        indx = len_new_doc[count]
        if indx is not None:
            avg_sim.append((indx, c, max_sim))


def process_two():
    global len_new_doc, len_temp_new_doc, len_temp_doc, len_doc, final_sim, avg_sim
    for n in len_new_doc:
        temp = 0
        added = None
        for a, b, c in avg_sim:
            if a == n:
                if c > temp:
                    temp = c
                    added = (a, b, c)
        if added is not None:
            final_sim.append(added)
    len_temp_new_doc = [q for q, w, e in final_sim]
    len_temp_doc = [t for r, t, y in final_sim]
    len_new_doc = [x for x in t1 if x not in len_temp_new_doc]
    len_doc = [y for y in t2 if y not in len_temp_doc]

# print(len(written_answers))
# print(len(new_list_mar))
for i in range(len(written_answers)):
    temp = new_list_mar[i]
    temp1 = new_list_ans[i]
    marking_scheme(temp)
    answer(temp1)
    pre_process()
    if len(documents) > len(new_documents):
        final_sim = []
        while len(len_new_doc) > 0:
            sim_array = []
            process_sim()
            avg_sim = []
            process_zero()
            process_one()
        dop = final_sim
        for g, h, l in dop:
            if l > 0.80:
                s1 = sentiment(documents[g])
                s2 = sentiment(new_documents[h])
                diff = abs(s1[0] - s2[0])
                if s1[0] < 0 and s2[0] < 0:
                    pass
                elif s1[0] > 0 and s2[0] > 0:
                    pass
                else:
                    if diff > 0.4:
                        final_sim.remove((g, h, l))

        sum1 = [j for g, h, j in final_sim]
        avg = (sum(sum1)) / len(documents)
        marks = round(((avg * 100) * int(marks_allotment[written_answers[i]])) / 100)
    else:
        final_sim = []
        while len(len_doc) > 0 and len(len_new_doc) > 0:
            sim_array = []
            process_sim1()
            avg_sim = []
            process_zero1()
            process_two()
        dop = final_sim
        for g, h, l in dop:
            if l > 0.80:
                s1 = sentiment(documents[h])
                s2 = sentiment(new_documents[g])
                diff = abs(s1[0] - s2[0])
                if s1[0] < 0 and s2[0] < 0:
                    pass
                elif s1[0] > 0 and s2[0] > 0:
                    pass
                else:
                    if diff > 0.4:
                        final_sim.remove((g, h, l))
        sum1 = [j for g, h, j in final_sim]
        avg = (sum(sum1)) / len(documents)
        marks = round(((avg * 100) * int(marks_allotment[written_answers[i]])) / 100)

    print(" Question", written_answers[i], "=", marks, "/", int(marks_allotment[written_answers[i]]))
    total_marks = total_marks + marks

kk = marks_allotment.keys()
ky = marks_allotment.values()

mar = [int(marks_allotment[d]) for d in kk if d in written_answers]
print("total marks out of", max_marks, "=", total_marks)

ttm = sum(mar)
summarks = total_marks


def get_value():
    s = "marks obtained: " + str(max_marks) + " / " + str(ttm)
    return s


 Question 1 = 25 / 50
 Question 2 = 0 / 50
 Question 3 = 50 / 50
 Question 4 = 50 / 50
 Question 5 = 50 / 50
total marks out of 250 = 175
